<a href="https://colab.research.google.com/github/Kandeo/Week-3-IP/blob/main/Christine_Kandeo_Python_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#     OVERVIEW
MTN Cote d'Ivoire would like to upgrade its technology infrastructure for its mobile users in Ivory Coast. 
Studying the given dataset, how does MTN Cote d'Ivoire go about the upgrade of its infrastructure strategy within the given cities?

The telecom data provided for this project is only a sample ( i.e. for only three days).  The data files for the project are as follow:

1. cells_geo_description.xlsx [Link] (Links to an external site.)
2. cells_geo.csv [Link] (Links to an external site.)
3. CDR_description.xlsx [Link] (Links to an external site.)
4. CDR 20120507 [http://bit.ly/TelecomDataset1] (Links to an external site.)
5. CDR 20120508 [http://bit.ly/TelecomDataset2] (Links to an external site.)
6. CDR 20120509 [http://bit.ly/TelecomDataset3]

Using Python for your analysis:

In [114]:
# Let's import the pandas library to be used:

import pandas as pd


# Cells_Geo

Loading Our Dataset

In [115]:
#To Load our Dataset:

#We start by downloading the csv file from the Provided link,
#Since the Document was on csv, i imported the Data From the Text file on Excel
#This generated the url='/content/Cells_geo.xlsx'

#Loading our Dataset and based on the cells_geo_description table:
url = '/content/Cells_geo.xlsx'
df = pd.read_excel(url, sheet_name='Sheet1', header=0, names= ['City', 'Status', 'Localisation', 'GeographicalZone', 'ZoneName', 'Longitude', 'Latitude', 'Region', 'Area', 'Cell_ID', 'Site_Code'])
df.head()

,City,Status,Localisation,GeographicalZone,ZoneName,Longitude,Latitude,Region,Area,Cell_ID,Site_Code
0,ADJAME,In Service,ABIDJAN,"""Abidjan_EST",AGBANNEW,-4.000667,5.368194,LAGUNE,NaN,fbb2e75af3,0bb492ef17
1,ADJAME,In Service,ABIDJAN,"""Abidjan_EST",AGBANNEW,-4.000667,5.368194,LAGUNE,NaN,1502501d78,0bb492ef17
2,ADJAME,In Service,ABIDJAN,"""Abidjan_EST",AGBANNEW,-4.000667,5.368194,LAGUNE,NaN,0c2e182a0c,0bb492ef17
3,ASSINIE,In Service,INTERIEUR,Grand-EST,"ASSINIE""",-3.488467,5.158761,SUD COMOE,NaN,69bdfc69fd,8a1e6ce31f
4,ASSINIE,In Service,INTERIEUR,Grand-EST,"ASSINIE""",-3.488467,5.158761,SUD COMOE,NaN,7a1029d2ab,8a1e6ce31f


Exploration

In [116]:
#To filter out cells and retain those whose Status is In Service:
df.loc[df["Status"] == 'In Service']    #Returns 3907 Rows

#To Filter based on the Location being Abidjan:
df.loc[df["Localisation"] == 'ABIDJAN']   #Returns 1669 Rows

#Grouping the Rows In Pandas:
#To group the Localisation first:
f = df.groupby('Localisation')
f
#To access each of this groups:
for Localisation, Localisation_df in f:
  print(Localisation)
  print(Localisation_df)

ABIDJAN
         City      Status Localisation  ...           Area     Cell_ID   Site_Code
0      ADJAME  In Service      ABIDJAN  ...            NaN  fbb2e75af3  0bb492ef17
1      ADJAME  In Service      ABIDJAN  ...            NaN  1502501d78  0bb492ef17
2      ADJAME  In Service      ABIDJAN  ...            NaN  0c2e182a0c  0bb492ef17
7       ABOBO  In Service      ABIDJAN  ...            NaN  a70c1d263b  56306cc7ca
8       ABOBO  In Service      ABIDJAN  ...            NaN  e2d04ea883  56306cc7ca
...       ...         ...          ...  ...            ...         ...         ...
3957  MARCORY  In Service      ABIDJAN  ...  ABIDJAN SOUTH  32dbfb021a  1ff98189c8
3958  MARCORY  In Service      ABIDJAN  ...  ABIDJAN SOUTH  71d9ff904d  1ff98189c8
3959  MARCORY  In Service      ABIDJAN  ...  ABIDJAN SOUTH  bbdd523132  1ff98189c8
3966  MARCORY  In Service      ABIDJAN  ...  ABIDJAN SOUTH  eb76e06e8e  6daf54117b
3967  MARCORY  In Service      ABIDJAN  ...  ABIDJAN SOUTH  e3d9d0ca54  6daf541

In [140]:
#To groupby city
g1 = df.groupby(["City"]).size().reset_index(name='Frequency')

#Finding the city with the largest number of frequency:
g1.nlargest(5, "Frequency")

#This are the most used cities:

,City,Frequency
122,COCODY,354
488,YOPOUGON,295
1,ABOBO,255
317,MARCORY,141
475,TREICHVILLE,132


In [101]:
#To view the number of cities:
set(df['City'])
len(set(df['City']))

#To view the number of Localisation:
set(df['Localisation'])
len(set(df['Localisation']))

2

Cleaning

In [50]:
#To check which observations are duplicates:
df.duplicated()

#To find missing data and delete them:
df.isnull()

#Dropping the missing values
df_no_missing = df.dropna()
df_no_missing.head()


,City,Status,Localisation,GeographicalZone,ZoneName,Longitude,Latitude,Region,Area,Cell_ID,Site_Code
23,ABOBO,In Service,ABIDJAN,Abidjan_NORD,ZONE E - Abidjan_NORD,-4.014139,5.4335,LAGUNE,ABIDJAN NORTH,f740406877,0f34c36ac5
24,ABOBO,In Service,ABIDJAN,Abidjan_NORD,ZONE E - Abidjan_NORD,-4.014139,5.4335,LAGUNE,ABIDJAN NORTH,959e54c8f7,0f34c36ac5
25,ABOBO,In Service,ABIDJAN,Abidjan_NORD,ZONE E - Abidjan_NORD,-4.014139,5.4335,LAGUNE,ABIDJAN NORTH,56b7e7000c,0f34c36ac5
26,ABOBO,In Service,ABIDJAN,Abidjan_NORD,ZONE E - Abidjan_NORD,-4.014139,5.4335,LAGUNE,ABIDJAN NORTH,bc51c17a6f,0f34c36ac5
27,ABOBO,In Service,ABIDJAN,Abidjan_NORD,ZONE E - Abidjan_NORD,-4.014139,5.4335,LAGUNE,ABIDJAN NORTH,e460975c9f,0f34c36ac5


In [143]:
#To Check which Columns have Null values
print(df.isnull().sum())    #Status and Area have Null values

City                 0
Status              67
Localisation         0
GeographicalZone     0
ZoneName             6
Longitude            0
Latitude             0
Region               0
Area                23
Cell_ID              0
Site_Code            0
dtype: int64


In [147]:
#To check null values in the Status column:
list(df.Status.unique())    #Has In Service and Null values

#To check in the Area Column:
list(df.Area.unique())    

[nan,
 'ABIDJAN NORTH',
 'NORTH WEST',
 'EAST',
 'ABIDJAN SOUTH',
 'SOUTH WEST',
 'CENTRE NORTH']

Analysis

In [54]:
#Summary Statistics for our Table:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3974 entries, 0 to 3973
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   City              3974 non-null   object 
 1   Status            3907 non-null   object 
 2   Localisation      3974 non-null   object 
 3   GeographicalZone  3974 non-null   object 
 4   ZoneName          3968 non-null   object 
 5   Longitude         3974 non-null   float64
 6   Latitude          3974 non-null   float64
 7   Region            3974 non-null   object 
 8   Area              3951 non-null   object 
 9   Cell_ID           3974 non-null   object 
 10  Site_Code         3974 non-null   object 
dtypes: float64(2), object(9)
memory usage: 372.6+ KB


,Longitude,Latitude
count,3974.000000,3974.000000
mean,-4.877659,6.244749
std,1.300771,1.304199
min,-8.503889,0.000000
25%,-5.846139,5.338639
50%,-4.084500,5.475000
75%,-3.993444,6.872306
max,0.000000,10.481861


In [60]:
#To Create a table of Localisation:
pd.pivot_table(df, index=['Localisation'] , aggfunc='count')


,Area,Cell_ID,City,GeographicalZone,Latitude,Longitude,Region,Site_Code,Status,ZoneName
Localisation,,,,,,,,,,
ABIDJAN,1650,1669,1669,1669,1669,1669,1669,1669,1636,1669
INTERIEUR,2301,2305,2305,2305,2305,2305,2305,2305,2271,2299


# CDR

Loading our Dataset

In [103]:
#Loading our CDR 20120507 File:

url = '/content/CDR 20120507.csv'
df7 = pd.read_csv('CDR 20120507.csv', header=0, names = ['Product', 'Value', 'Date_Time', 'Cell_On_Site', 'DW_A_Number_int', 'DW_B_Number_int', 'Country_A', 'Country_B', 'Cell_ID', 'Site ID']) 
df7.head(5)

#Based on the CDR Description file, 
#Column Cell_On_Site, Country_A and Country_B are not needed, 
#Dropping the 3 columns:

df7.drop(['Cell_On_Site', 'Country_A', 'Country_B'], axis=1, inplace=True)
df7.head()

#Finding Unique Values in our data:
df7.Product.unique()

#Identifying any Duplicates:
df7.duplicated()

#Grouping our Products as per the Unique Values:
f= df7.groupby('Product')
f
#To access each of this groups:
#for Product, Product_df in f:
#  print(Product)
#  print(Product_df)

#To access a specific Product:
#f.get_group('Voice')

#To find the Sum of the Products:
f.sum()


,Value
Product,
Voice,108869
data,7032
sms,13473


In [64]:
#Summary Statistics of 20120507:
f.describe()

Value                                                     
          count       mean         std  min  25%   50%   75%     max
Product                                                             
Voice    2219.0  49.062190  114.668144  0.0  1.0  20.0  53.0  1860.0
data      807.0   8.713755  158.285506  0.0  0.0   0.0   0.0  4440.0
sms      1975.0   6.821772   15.060924  0.0  0.0   0.0   0.0   100.0

In [104]:
#Loading our CDR 20120508 File
#And Based on the CDR Description,

url = '/content/CDR 20120508.csv'
df8 = pd.read_csv('CDR 20120508.csv', header=0, names = ['Product', 'Value', 'Date_Time', 'Cell_On_Site', 'DW_A_Number_int', 'DW_B_Number_int', 'Country_A', 'Country_B', 'Cell_ID', 'Site ID']) 
df8.head(5)

#Based on the CDR Description file, 
#Dropping: Column Cell_On_Site, Country_A and Country_B,

df8.drop(['Cell_On_Site', 'Country_A', 'Country_B'], axis=1, inplace=True)
df8.head()

#Finding Unique Values in our data:
df8.Product.unique()

#Identifying any Duplicates:
df8.duplicated()

#Grouping our Products as per the Unique Values:
f= df8.groupby('Product')
f
#To access each of this groups:
#for Product, Product_df in f:
#  print(Product)
#  print(Product_df)

#To access a specific Product:
f.get_group('sms')

#To find the Sum of the Products:
f.sum()

,Value
Product,
Voice,103486
data,5500
sms,12713


In [66]:
#Summary Statistics of 20120508:
f.describe()

Value                                                      
          count       mean         std  min  25%   50%    75%     max
Product                                                              
Voice    1906.0  54.294858  147.777044  0.0  1.0  20.0  52.75  3380.0
data      766.0   7.180157   58.972256  0.0  0.0   0.0   0.00  1000.0
sms      2329.0   5.458566   14.291520  0.0  0.0   0.0   0.00   100.0

In [105]:
#Loading our CDR 20120509 File:

url = '/content/CDR 20120509.csv'
df9 = pd.read_csv('CDR 20120509.csv', header=0, names = ['Product', 'Value', 'Date_Time', 'Cell_On_Site', 'DW_A_Number_int', 'DW_B_Number_int', 'Country_A', 'Country_B', 'Cell_ID', 'Site ID']) 
df9.head(5)

#Based on the CDR Description file, 
#Dropping Column Cell_On_Site, Country_A and Country_B,

df9.drop(['Cell_On_Site', 'Country_A', 'Country_B'], axis=1, inplace=True)
df9.head()

#Finding Unique Values in our data:
df9.Product.unique()

#Identifying any Duplicates:
df9.duplicated()

#Grouping our Products as per the Unique Values:
f= df9.groupby('Product')
f
#To access each of this groups:
#for Product, Product_df in f:
# print(Product)
# print(Product_df)

#To access a specific Product:
f.get_group('sms')

#To find the Sum of the Products:
f.sum()

#This is the value of the different products on the 3rd day

,Value
Product,
Voice,100238
data,4755
sms,27241


In [70]:
#Summary Statistics of 20120509:
f.describe()

Value                                                      
          count       mean         std  min  25%   50%   75%      max
Product                                                              
Voice    1541.0  65.047372  399.156235  0.0  1.0  20.0  53.0  12900.0
data      693.0   6.861472   54.666470  0.0  0.0   0.0   0.0    832.0
sms      2767.0   9.844958   14.412532  0.0  0.0   0.0  25.0    100.0

Exploring and Cleaning

In [106]:
#Merging the Datasets for the 3 days:
dfCombined = pd.concat([df7,df8,df9], axis=0)
dfCombined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15003 entries, 0 to 5000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Product          15003 non-null  object
 1   Value            15003 non-null  int64 
 2   Date_Time        15003 non-null  object
 3   DW_A_Number_int  15003 non-null  object
 4   DW_B_Number_int  15003 non-null  object
 5   Cell_ID          15003 non-null  object
 6   Site ID          13006 non-null  object
dtypes: int64(1), object(6)
memory usage: 937.7+ KB


In [107]:
#Changing the data type of the Date time Column:
dfCombined['Date_Time'] = pd.to_datetime(dfCombined['Date_Time'])
dfCombined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15003 entries, 0 to 5000
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Product          15003 non-null  object        
 1   Value            15003 non-null  int64         
 2   Date_Time        15003 non-null  datetime64[ns]
 3   DW_A_Number_int  15003 non-null  object        
 4   DW_B_Number_int  15003 non-null  object        
 5   Cell_ID          15003 non-null  object        
 6   Site ID          13006 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 937.7+ KB


In [108]:
#Running our Combined Table:
dfCombined

,Product,Value,Date_Time,DW_A_Number_int,DW_B_Number_int,Cell_ID,Site ID
0,Voice,0,2012-05-06 23:04:37,49fae048b547780,1a49f8b0ded9de6,/TJNe+Mmtu,/TJNe+Mmtu
1,sms,0,2012-05-06 23:05:25,7a54a6eaee19e65,93343ba6e41af95,/+cKZKIp41,/+cKZKIp41
2,sms,25,2012-05-06 23:05:41,6008887b0207d82,7aacc9709ce0a20,/u0FSD+ahi,/u0FSD+ahi
3,Voice,56,2012-05-06 23:05:42,7370032e20f4e96,6af4a60e74c91d3,+cNeJzsTp3,+cNeJzsTp3
4,Voice,14,2012-05-06 23:06:29,966ba4656e38e62,02aa736ee555d5c,051KhYDCpv,051KhYDCpv
...,...,...,...,...,...,...,...
4996,Voice,0,2012-05-09 00:01:43,467333ade2b4b9c,0ab2acd804d01fe,2aa04dcab7,f7d63a29e5
4997,Voice,62,2012-05-09 00:01:43,e077a6b74c5053c,4c8098eb6ea1a41,7f1f2f889f,b4ddb9e93c
4998,Voice,32,2012-05-09 00:01:43,77f53c39d27c050,9d064f013e2cab2,a4731e780e,d68260ef50
4999,Voice,30,2012-05-09 00:01:43,672ce74c418aa7a,f9ff5af59cf0d68,98ed726da2,b4130b54ab


In [110]:
dfCombined.groupby('Cell_ID').count().nlargest(5,'Product')

,Product,Value,Date_Time,DW_A_Number_int,DW_B_Number_int,Site ID
Cell_ID,,,,,,
ffa6759bb2,1996,1996,1996,1996,1996,0
03ee801971,75,75,75,75,75,75
cc7ce19d6b,65,65,65,65,65,65
d22224b2fa,43,43,43,43,43,43
b78724d6fc,42,42,42,42,42,42


Analysis

In [79]:
#To find the total sum for all the days:
f= dfCombined.groupby('Product')
f
#To access each of this groups:
#for Product, Product_df in f:
#  print(Product)
#  print(Product_df)

#To find the Sum of the Products:
f.sum()

#Most used product for the 3 days was Voice

,Value
Product,
Voice,312593
data,17287
sms,53427


In [80]:
f.describe()

Value                                                      
          count       mean         std  min  25%   50%   75%      max
Product                                                              
Voice    5666.0  55.169961  236.315367  0.0  1.0  20.0  53.0  12900.0
data     2266.0   7.628861  104.899256  0.0  0.0   0.0   0.0   4440.0
sms      7071.0   7.555791   14.680076  0.0  0.0   0.0  25.0    100.0

In [150]:
# Create a table of the products by date time 
pd.pivot_table(dfCombined, index=['Product','Date_Time'], aggfunc='sum')

Value
Product Date_Time                 
Voice   2012-05-06 23:04:37      0
        2012-05-06 23:05:42     56
        2012-05-06 23:06:29     14
        2012-05-06 23:08:02      0
        2012-05-06 23:08:03      0
...                            ...
sms     2012-05-09 00:01:39    200
        2012-05-09 00:01:40    125
        2012-05-09 00:01:41    170
        2012-05-09 00:01:42    125
        2012-05-09 00:01:43    125

[2933 rows x 1 columns]

In [161]:
#To group the Site ID by their value:

dfCombined.groupby('Site ID')['Value'].sum()

Site ID
++ctND+jGD    134
+38u2u/rfx    247
+854AcBQT2    154
+Fs4mTvdKx     31
+QZ9qRtNxA     55
             ... 
feffc8c92b    297
ff1b78ec81    322
ffa6bdc032    499
ffc9f56819    177
fff5ac6e3f     28
Name: Value, Length: 1357, dtype: int64

In [113]:
#Trying to merge our two tables from the Cells_geo and the df Combined
#on the cell_ID column 

inner_merged = pd.merge(df,dfCombined)
inner_merged

#this does not yield showing that the two columns dont have similar details on the Cell_ID.

,City,Status,Localisation,GeographicalZone,ZoneName,Longitude,Latitude,Region,Area,Cell_ID,Site_Code,Product,Value,Date_Time,DW_A_Number_int,DW_B_Number_int,Site ID
